In [5]:
import pystan
from pystan import StanModel

import numpy as np
import numpy.random as rd
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation as ani
import matplotlib.cm as cm
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)

from sklearn import cross_validation


/home/watanabe/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
kilo10000 = pd.read_csv("../data/kilo10000_interpolate.csv")
kilo10001 = pd.read_csv("../data/kilo10001_interpolate.csv")
kilo10002 = pd.read_csv("../data/kilo10002_interpolate.csv")
kilo10003 = pd.read_csv("../data/kilo10003_interpolate.csv")
kilo10004 = pd.read_csv("../data/kilo10004_interpolate.csv")
data = pd.concat((kilo10000["高低左"],kilo10001["高低左"],kilo10002["高低左"],kilo10003["高低左"],kilo10004["高低左"]), axis=1)
data.columns = ["10000","10001","10002","10003","10004"]
data.index = pd.to_datetime(kilo10000["date"])
data.head()

,10000,10001,10002,10003,10004
date,,,,,
2017-04-01,-1.16,-1.17,-1.09,-0.64,0.470
2017-04-02,-0.81,-0.83,-0.71,-0.15,1.010
2017-04-03,-0.99,-0.91,-0.67,-0.02,1.220
2017-04-04,-0.98,-0.81,-0.50,0.18,1.385
2017-04-05,-0.97,-0.71,-0.33,0.38,1.550


In [19]:
data["2017"]

,10000,10001,10002,10003,10004
date,,,,,
2017-04-01,-1.160000,-1.170,-1.090000,-0.640,0.470
2017-04-02,-0.810000,-0.830,-0.710000,-0.150,1.010
2017-04-03,-0.990000,-0.910,-0.670000,-0.020,1.220
2017-04-04,-0.980000,-0.810,-0.500000,0.180,1.385
2017-04-05,-0.970000,-0.710,-0.330000,0.380,1.550
2017-04-06,-1.480000,-1.070,-0.290000,0.950,2.630
2017-04-07,-0.660000,-0.560,-0.460000,-0.100,0.820
2017-04-08,-1.290000,-0.980,-0.560000,0.150,1.300
2017-04-09,-0.980000,-0.820,-0.620000,-0.130,0.870


In [25]:
# 訓練データ, テストデータに分割する
#https://note.nkmk.me/python-pandas-time-series-datetimeindex/
train = data["2017/4/1":"2017/12/31"]
test = data["2018/1/1":]
print ([d.shape for d in [train, test]])

[(275, 5), (90, 5)]


In [ ]:
model = """


"""